# ResNet101_ASPP

## 1. Environment Setup
### 1.0. Check GPU

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


### 1.1. Install Pytorch and torchvision

In [2]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### 1.2. Install scipy

In [3]:
!pip install scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### 1.3. Install PyDenseCRF

In [4]:
!pip install pydensecrf

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### 1.4. Import

In [5]:
import augmentation, model, train

import torch
import torchvision

# Ignore Warning
import warnings
warnings.filterwarnings(action='ignore')

## 2. Data Preprocessing

In [6]:
train_batch_size = 10
test_batch_size = 1

In [7]:
train_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), 
                                             augmentation.RandomResizedCrop((256, 256), scale=(1., 1.), ratio=(0.5, 1.5)), 
                                             augmentation.RandomHorizontalFlip(),
                                             augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), 
                                           augmentation.Resize(256),
                                           augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = torchvision.datasets.SBDataset(root='./', image_set='train_noval', mode='segmentation', download=False, transforms=train_tf)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_dataset = torchvision.datasets.VOCSegmentation(root='./', year='2012', image_set='val', download=False, transforms=val_tf)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True)

## 3. Train Model

In [8]:
num_classes = 21
atrous_rates = [6, 12, 18, 24]
scales = [0.5, 0.75, 1]
ignore_index = 255

gpu_id = 7
print_freq = 200
epoch_print = 10

save = True
epochs = 150

lr = 0.001
momentum = 0.9
weight_decay = 0.0005

In [9]:
MSC_ResNet101_ASPP = train.MSC_ResNet101_ASPP(num_classes=num_classes, atrous_rates=atrous_rates, 
                                              scales=scales, ignore_index=ignore_index,
                                              gpu_id=gpu_id, print_freq=print_freq, epoch_print=epoch_print)

In [10]:
MSC_ResNet101_ASPP.train(train_loader, val_loader, save=save, epochs=epochs, 
                         lr=lr, momentum=momentum, weight_decay=weight_decay)

Epoch 1 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 200 - Train Loss : 0.479254, Test Loss : 0.419549, Test mIoU : 54.5784
Saved Best Model


*********************************** Best mIoU Updated ***********************************
Iteration : 400 - Train Loss : 0.378670, Test Loss : 0.338870, Test mIoU : 59.1379
Saved Best Model

Epoch 11 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 200 - Train Loss : 0.073195, Test Loss : 0.246744, Test mIoU : 70.6672
Saved Best Model


*********************************** Best mIoU Updated ***********************************
Iteration : 400 - Train Loss : 0.161293, Test Loss : 0.241482, Test mIoU : 71.2737
Saved Best Model

Epoch 21 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 200 - Train Loss : 0.126879, Test Loss : 0.248051, Test mIoU : 71.8538